In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# from torch.utils.tensorboard import SummaryWriter
from tensorboardX import SummaryWriter
from torchsummary import summary

import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm

from Model import *

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./Dataset', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=2)

In [ ]:
test_dataset = torchvision.datasets.CIFAR10(root='./Dataset', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# net = VGG('VGG11')
# net = SimpleDLA()
net = MobileNetV2()
# net = RegNetX_200MF()

In [ ]:
net = net.to(device)
summary(net, input_size=(3,32, 32))

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
def valid(epoch):
    net.eval()
    loss_list, p_list, y_list = [], [], []
    for idx, (x, y) in enumerate(test_loader):
        x, y = x.to(device), y.to(device)
        p = net(x)

        loss = criterion(p, y)
        loss_list.append(loss.item())

        _, p = p.max(axis=-1)
        y_list.append(y.detach().cpu().numpy())
        p_list.append(p.detach().cpu().numpy())
    
    loss = np.array(loss_list).mean()
    acc = (np.array(p_list) == np.array(y_list)).astype(np.float32).mean()
    
    return loss, acc

In [ ]:
def train(epoch):
    net.train()        
    loss_list, p_list, y_list = [], [], []
    with tqdm((enumerate(train_loader)), desc='epoch%3d'%epoch, total=len(train_loader), ncols=0) as t:
        for idx, (x, y) in t:
            x, y = x.to(device), y.to(device)
            p = net(x)
            
            optimizer.zero_grad()
            loss = criterion(p, y)
            loss.backward()
            optimizer.step()

            loss_list.append(loss.item())

            _, p = p.max(axis=-1)
            y_list.append(y.detach().cpu().numpy())
            p_list.append(p.detach().cpu().numpy())
            if idx+1 < len(train_loader):
                t.set_postfix({'loss':'%0.4f'%loss_list[-1]})
            else:
                loss = np.array(loss_list).mean()
                acc = (np.array(p_list) == np.array(y_list)).astype(np.float32).mean()
                
                valid_loss, valid_acc = valid(epoch)
                t.set_postfix({'loss':'%0.4f'%loss, 'acc':'%0.4f'%acc, 'valid_loss':'%0.4f'%valid_loss, 'valid_acc':'%0.4f'%valid_acc})
        return loss, acc, valid_loss, valid_acc

In [ ]:
writer = SummaryWriter()
# writer.add_graph(net, input_to_model=torch.from_numpy(np.random.randn(2,3,32,32).astype(np.float32)).to(device), verbose=False)
for epoch in range(200):
    train_loss, train_acc, valid_loss, valid_acc = train(epoch)
    writer.add_scalar('loss/train', train_loss, epoch)
    writer.add_scalar('acc/train', train_acc, epoch)
    writer.add_scalar('loss/valid', valid_loss, epoch)
    writer.add_scalar('acc/valid', valid_acc, epoch)